<a href="https://colab.research.google.com/github/danielkim717/3-2/blob/main/%EC%98%88%EB%B9%84%EC%BA%A1%EC%8A%A4%ED%86%A4_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy pandas matplotlib scikit-learn torch torchvision

In [ ]:
from google.colab import files
uploaded = files.upload()  # 파일 선택 → WM811K.pkl 업로드

In [ ]:
#데이터 로드 및 분할

import pandas as pd

# 업로드한 pkl 불러오기
df = pd.read_pickle("/content/WM811K.pkl")

print(df.head())
print("train/val/test 분포:", df["trainTestLabel"].value_counts())


In [ ]:
#데이터셋&데이터로더

# ===== Colab 전용: WM-811K DataFrame 버전 → Dataset/DataLoader 통합 셋업 =====
# 1) 필요한 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
import sys
import subprocess
import random
import math
import matplotlib.pyplot as plt
from typing import Dict

warnings.filterwarnings("ignore", category=DeprecationWarning)
print("CUDA available:", torch.cuda.is_available())

# 2) WM811K.pkl 업로드 (한 번만 업로드하면 /content/ 에 존재)
from google.colab import files
print("👇 WM811K.pkl 파일을 선택해서 업로드하세요")
try:
    uploaded = files.upload()
    PKL_PATH = "/content/WM811K.pkl"
except FileNotFoundError:
    print("WM811K.pkl 파일이 이미 존재하거나 업로드되지 않았습니다. 기존 경로를 사용합니다.")
    PKL_PATH = "/content/WM811K.pkl"
except Exception as e:
    print(f"파일 업로드 중 오류가 발생했습니다: {e}")
    sys.exit()

# 3) 로드 & 정규화(라벨/스플릿이 배열/대소문자 섞여도 안전)
try:
    df = pd.read_pickle(PKL_PATH)
    print("DataFrame shape:", df.shape)
    print("Columns:", df.columns.tolist())
except Exception as e:
    print(f"파일 로드 중 오류가 발생했습니다: {e}")
    sys.exit()

def to_scalar_str(v):
    if isinstance(v, (list, tuple, np.ndarray)):
        a = np.array(v)
        if a.size == 1:
            try: v = a.item()
            except Exception: v = a.reshape(-1)[0]
        else:
            v = "|".join(map(str, a.reshape(-1).tolist()))
    return str(v)

# 필수 컬럼 정규화
assert "waferMap" in df.columns, "waferMap 컬럼이 필요합니다."
assert "failureType" in df.columns, "failureType 컬럼이 필요합니다."
df["failureType"] = df["failureType"].apply(to_scalar_str).str.strip()
if "trainTestLabel" in df.columns:
    df["trainTestLabel"] = df["trainTestLabel"].apply(to_scalar_str).str.strip()

# 정상 라벨 표준화
df["failureType"] = df["failureType"].replace(
    {"None":"none","NONE":"none","Null":"none","null":"none"}
)

# 스플릿 표준화 (training / test)
def normalize_split(v: str) -> str:
    lv = str(v).lower()
    if "train" in lv: return "training"
    if "test"  in lv: return "test"
    if lv in {"0","1"}: return "training" if lv=="0" else "test"
    return lv
if "trainTestLabel" in df.columns:
    df["trainTestLabel"] = df["trainTestLabel"].apply(normalize_split)

print("Unique failureType (sample 20):", sorted(df["failureType"].unique())[:20])
if "trainTestLabel" in df.columns:
    print("Split uniques:", sorted(df["trainTestLabel"].unique()))

# 4) 제공 스플릿 그대로 사용 + train→val 분할
SEED = 42
if "trainTestLabel" in df.columns:
    df_train_all = df[df["trainTestLabel"]=="training"].copy()
    df_test      = df[df["trainTestLabel"]=="test"].copy()
    if len(df_train_all)==0:
        raise RuntimeError("training 표기가 0개입니다. 위 출력의 Split uniques를 확인하고 normalize_split 로직을 조정하세요.")
    df_train, df_val = train_test_split(
        df_train_all, test_size=0.15, random_state=SEED, stratify=df_train_all["failureType"]
    )
else:
    # 만약 스플릿이 없으면 8:1:1로 생성
    tr_all, df_test = train_test_split(df, test_size=0.1, random_state=SEED, stratify=df["failureType"])
    df_train, df_val = train_test_split(tr_all, test_size=0.111, random_state=SEED, stratify=tr_all["failureType"])  # 0.111*0.9 ≈ 0.1

print(f"[SPLIT] train={len(df_train)}, val={len(df_val)}, test={len(df_test)}")

# 5) 라벨 매핑
classes = sorted(df["failureType"].unique().tolist())
label2idx = {c:i for i,c in enumerate(classes)}
idx2label = {i:c for c,i in label2idx.items()}
print(f"#classes = {len(classes)}")

# 6) 전처리/증강 보조 함수
def to_tensor_1ch(img):
    """
    numpy/image-like → torch.FloatTensor [1,H,W], 값 0~1 정규화
    """
    a = np.array(img)
    if a.ndim == 3 and a.shape[-1] == 1:
        a = a[...,0]
    a = a.astype(np.float32)
    # 0~1 스케일
    amin, amax = float(a.min()), float(a.max())
    a = (a - amin) / (amax - amin + 1e-8) if amax > amin else np.zeros_like(a, dtype=np.float32)
    t = torch.from_numpy(a).float().unsqueeze(0)  # [1,H,W]
    return t

def resize_tensor_bilinear(t, size=128):
    return F.interpolate(t.unsqueeze(0), size=(size, size), mode="bilinear", align_corners=False).squeeze(0)

def random_geom_aug(t):
    # 90° 회전 + 좌우/상하 flip (웨이퍼 대칭)
    k = random.choice([0,1,2,3])
    t = torch.rot90(t, k, dims=[1,2])
    if random.random() < 0.5:
        t = torch.flip(t, dims=[2])
    if random.random() < 0.5:
        t = torch.flip(t, dims=[1])
    return t

# 7) WaferDataset
class WaferDataset(Dataset):
    def __init__(self, df: pd.DataFrame, label2idx: Dict[str,int], train: bool = True, img_size: int = 128):
        self.df = df.reset_index(drop=True)
        self.label2idx = label2idx
        self.train = train
        self.img_size = img_size

    def __len__(self): return len(self.df)

    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = row["waferMap"]
        y   = self.label2idx[str(row["failureType"])]
        t = to_tensor_1ch(img)
        t = resize_tensor_bilinear(t, self.img_size)
        if self.train:
            t = random_geom_aug(t)
        return t, y

# 8) DataLoader (+불균형 대응 WeightedRandomSampler)
IMG_SIZE = 128
BATCH  = 128

train_ds = WaferDataset(df_train, label2idx, train=True,  img_size=IMG_SIZE)
val_ds   = WaferDataset(df_val,   label2idx, train=False, img_size=IMG_SIZE)
test_ds  = WaferDataset(df_test,  label2idx, train=False, img_size=IMG_SIZE)

# 클래스 가중치 (반비례 가중) + 샘플러
vc = df_train["failureType"].value_counts()
w = np.array([1.0 / max(vc.get(lbl,1), 1) for lbl in classes], dtype=np.float32)
w = w / w.mean()
class_weight = torch.tensor(w, dtype=torch.float32)

sample_weights = df_train["failureType"].map(lambda s: class_weight[label2idx[s]].item()).values
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_ds, batch_size=BATCH, sampler=sampler, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False,   num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False,   num_workers=2, pin_memory=True)

print("[OK] DataLoaders ready.")

# 9) 빠른 무결성 체크(한 배치 꺼내 보기)
imgs, labels = next(iter(train_loader))
print("Batch image tensor shape:", imgs.shape)
print("Batch labels shape:", labels.shape)

# 라벨 일부를 실제 이름으로 매핑해보기
idx2label = {v:k for k,v in label2idx.items()}
print("Sample labels:", [idx2label[int(l)] for l in labels[:10]])

# 10) (선택) 배치 시각화 12장
def show_grid(tensor_batch, label_batch, idx2label, cols=6):
    n = min(len(tensor_batch), cols*2)
    rows = math.ceil(n/cols)
    plt.figure(figsize=(cols*2, rows*2))
    for i in range(n):
        plt.subplot(rows, cols, i+1)
        plt.imshow(tensor_batch[i,0].cpu().numpy(), origin="lower", interpolation="nearest")
        plt.title(idx2label[int(label_batch[i])], fontsize=8)
        plt.xticks([]); plt.yticks([])
    plt.tight_layout(); plt.show()

show_grid(imgs, labels, idx2label, cols=6)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WaferCNN(nn.Module):
    def __init__(self, num_classes, input_size=(1, 128, 128)):
        super(WaferCNN, self).__init__()

        # 첫 번째 컨볼루션 블록 (Conv -> ReLU -> BatchNorm -> Pool)
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32), # 배치 정규화 추가
            nn.MaxPool2d(2, 2),

            # 두 번째 컨볼루션 블록
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64), # 배치 정규화 추가
            nn.MaxPool2d(2, 2)
        )

        # 동적으로 fc1 입력 차원 계산
        dummy_input = torch.zeros(1, *input_size)
        conv_output_size = self._get_conv_output_size(dummy_input)

        # 분류기 (Classifier)
        self.classifier = nn.Sequential(
            nn.Linear(conv_output_size, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5), # 드롭아웃 추가 (일반적으로 0.5를 많이 사용)
            nn.Linear(128, num_classes)
        )

    def _get_conv_output_size(self, x):
        """
        입력 이미지 크기에 따라 컨볼루션 레이어 출력 크기를 동적으로 계산하는 헬퍼 함수
        """
        with torch.no_grad(): # 계산 그래프를 만들 필요가 없으므로 효율적
            x = self.features(x)
            n_size = x.view(x.size(0), -1).size(1)
        return n_size

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
import torch.optim as optim
import numpy as np

# ===== 1. 설정 및 초기화 =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 모델, 손실 함수, 옵티마이저 초기화
model = WaferCNN(num_classes=len(label2idx)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 학습률 스케줄러 (검증 손실이 개선되지 않을 때 학습률 감소)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

# 최적 모델 저장을 위한 변수 초기화
best_val_loss = float('inf')
num_epochs = 10 # 에포크 수를 10으로 설정 (조절 가능)

# ===== 2. 학습 및 검증 루프 =====
for epoch in range(num_epochs):
    # 훈련 단계
    model.train() # 모델을 훈련 모드로 설정
    train_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_loader.dataset)

    # 검증 단계
    model.eval() # 모델을 평가 모드로 설정
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad(): # 검증 단계에서는 기울기 계산 비활성화
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_loss /= len(val_loader.dataset)
    val_accuracy = 100 * correct / total

    # 학습률 스케줄러 업데이트
    scheduler.step(val_loss)

    # 결과 출력
    print(f'Epoch {epoch+1}/{num_epochs} | '
          f'Train Loss: {train_loss:.4f} | '
          f'Val Loss: {val_loss:.4f} | '
          f'Val Accuracy: {val_accuracy:.2f}%')

    # 최적 모델 저장
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_wafer_model.pth')
        print(f'-> Model saved! Best validation loss: {best_val_loss:.4f}')

print("Training finished.")

In [ ]:
import torch
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 모델을 평가 모드로 설정
model.eval()

# 예측값과 실제 라벨을 저장할 리스트 초기화
all_preds, all_labels = [], []

# 추론 시 기울기 계산 비활성화 (메모리 절약 및 속도 향상)
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        # 예측값과 라벨을 CPU로 이동 후 리스트에 추가
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 분류 리포트 출력
print("분류 리포트:")
print(classification_report(all_labels, all_preds, target_names=list(label2idx.keys())))

# 혼동 행렬 생성 및 시각화
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(label2idx.keys()), yticklabels=list(label2idx.keys()))
plt.title('혼동 행렬 (Confusion Matrix)')
plt.xlabel('예측 라벨 (Predicted Label)')
plt.ylabel('실제 라벨 (True Label)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()